# Code
Für den CI Overlap cio
# Imports

In [1]:
import pandas as pd
from pathlib import Path
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Optional, Tuple, Union
import seaborn as sns
import matplotlib.pyplot as plt
from syntheval import SynthEval

# User Configuration

In [2]:
SAVE_CSV = True  # Save figures

REAL_FILE = "20250301_data_20250510_122405_final_100_train.csv"
SYNTH_FILE = "20250301_data_20250510_122405_final_100_synth.csv"
HOLDOUT_FILE = "20250301_data_20250510_122405_final_100_holdout.csv"

DATA_DIR = Path("../../data")
OUTPUT_DIR_CSV = Path("results")
OUTPUT_DIR_CSV.mkdir(parents=True, exist_ok=True)  # ensure output folder exists

# Set my design
sns.set_theme(
    style="white",
    context="talk",
    palette="colorblind"
)

# Utility Functions

## Load Data & Assign Correct Datatypes

In [3]:
def convert_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Konvertiert bestimmte Spalten des DataFrames in die gewünschten Datentypen:
     - definierte Spalten als category
     - definierte Spalten als pandas Nullable Integer (Int64)
     - consciousness_level und news_score als geordnete Categoricals
    """
    df = df.copy()  # Änderungen nicht am Original vornehmen

    # 1) Kategorische Spalten
    cat_cols = ['gender', 'ethnicity', 'chief_complaint', 'icd_block']
    for col in cat_cols:
        df[col] = df[col].astype('category')

    # 2) Integer-Spalten mit Nullable Integer dtype
    int_cols = ['age', 'systolic_bp', 'diastolic_bp',
                'heart_rate', 'respiratory_rate', 'oxygen_saturation']
    for col in int_cols:
        df[col] = df[col].astype('Int64')

    # 3) Geordnete Categoricals
    df['consciousness_level'] = pd.Categorical(
        df['consciousness_level'],
        categories=['A', 'C', 'V', 'P', 'U'],
        ordered=True
    )
    df['news_score'] = pd.Categorical(
        df['news_score'],
        categories=list(range(19)),
        ordered=True
    )

    return df

def load_data(
    real_filename: Union[str, Path],
    synth_filename: Optional[Union[str, Path]] = None,
    holdout_filename: Optional[Union[str, Path]] = None,
    data_dir: Path = DATA_DIR
) -> Tuple[pd.DataFrame, Optional[pd.DataFrame], Optional[pd.DataFrame]]:
    """
    Lädt die realen, synthetischen und optionalen Holdout-CSV-Dateien
    aus data_dir und wandelt sie über convert_dtypes um.

    Returns:
        df_real: pd.DataFrame
        df_synth: Optional[pd.DataFrame]
        df_holdout: Optional[pd.DataFrame]
    """
    def _read_and_convert(fn: Union[str, Path]) -> pd.DataFrame:
        return (
            pd.read_csv(data_dir / fn, low_memory=False)
              .pipe(convert_dtypes)
        )

    df_real    = _read_and_convert(real_filename)
    df_synth   = _read_and_convert(synth_filename)   if synth_filename   else None
    df_holdout = _read_and_convert(holdout_filename) if holdout_filename else None

    return df_real, df_synth, df_holdout

# Main Routine
## Daten Laden

In [4]:
df_real, df_synth, df_holdout = load_data(
    REAL_FILE,
    SYNTH_FILE,
    HOLDOUT_FILE,
    data_dir=DATA_DIR
)

In [5]:
num_attrs = df_real.select_dtypes(include=['number']).columns
cat_attrs = df_real.select_dtypes(exclude=['number']).columns

In [6]:
# Laufzeit: 3m 17.8s
S = SynthEval(df_real, cat_cols=cat_attrs, verbose=False)
res = S.evaluate(df_synth, cio={})

In [7]:
S._raw_results['cio']
# S._raw_results['cio']['frac non-overlaps']


{'avg overlap': 0.07657454068263034,
 'overlap err': 0.07657454068263031,
 'num non-overlaps': 6,
 'frac non-overlaps': 0.8571428571428571}

In [8]:
test = df_real[['age']].copy()
print(test.head())

   age
0   71
1   91
2   87
3   87
4   83


In [9]:
results = []

for attr in num_attrs:
    result = []
    r = df_real[[attr]].copy()
    s = df_synth[[attr]].copy()
    S = SynthEval(r, verbose=False)
    _ = S.evaluate(s, cio={})
    # result = S._raw_results['cio']['avg overlap'].copy()
    # result['feature'] = attr
    # results.append(result)
    results.append({'feature': attr, 'avg_ci_overlap': S._raw_results['cio']['avg overlap']})

results = pd.DataFrame(results)
if SAVE_CSV:
    results.to_csv(OUTPUT_DIR_CSV / "cio_results.csv", index=False, sep=";", decimal=",", encoding="utf-8")
print(results)
print(results['avg_ci_overlap'].mean())

             feature  avg_ci_overlap
0                age        0.000000
1        temperature        0.536022
2         heart_rate        0.000000
3   respiratory_rate        0.000000
4  oxygen_saturation        0.000000
5        systolic_bp        0.000000
6       diastolic_bp        0.000000
0.07657454068263034


c:\Masterarbeit\SynthEval\syntheval\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Masterarbeit\SynthEval\syntheval\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Masterarbeit\SynthEval\syntheval\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Masterarbeit\SynthEval\syntheval\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Masterarbeit\SynthEval\syntheval\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Masterarbeit\SynthEval\syntheval\Lib\site-packages\numpy\core\_me